In [ ]:
!pip install seaborn
!pip install causalgraphicalmodels
!pip install matplotlib
!pip install -U scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import causalgraphicalmodels
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import tensorflow as tf

# Model Building


In [ ]:
def sigmoid(x):
    return (1 / (1 + math.exp(-x)))
def epsilon_z():
    return np.random.normal(loc = 0,scale = 1)
def epsilon_x():
    return np.random.normal(loc = 0,scale = 1)
def epsilon_y():
    return np.random.normal(loc = 0,scale = 1)
def para_c1():
    return np.random.normal(loc = -.8,scale = 2)
def para_c2():
    return np.random.normal(loc = 1.2,scale = 1)
def para_c3():
    return np.random.normal(loc = 1,scale = 2)

def build_z1():
    return sigmoid(epsilon_z())
def build_z2(z1):
    return sigmoid(-.5*(2*z1-1)+epsilon_z())
def build_z3(z1):
    return sigmoid(2*z1-1 + epsilon_z())
def build_x(z1,z2,u):
    return sigmoid((2*z1-1)*u - (2*z2-1)*u + epsilon_x())
def build_z4(z2,z3,x):
    return sigmoid(para_c1()*z2*x - para_c2()*z3*x + epsilon_z())
def build_y(z1,z3,u,sum):
    return sigmoid(sum + 2*z1 + 2*z3 -2 + 1.5*u + epsilon_y())


fZ3 (Z1) = B (Sigm (2Z1 − 1) + epsilonz3)

In [ ]:
def build_causal_model(N,d):
    scm_np = np.empty((N,d+5))
    col_name = list()
    sum = 0.0
    for i in range(N):
        if(i == 0):
            col_name.extend(['z1','z2','z3','x'])
        u = np.random.normal(loc = 0, scale = 2)
        z1 = np.random.binomial(n=1,p=build_z1())
        z2 = np.random.binomial(n=1,p=build_z2(z1))
        z3 = np.random.binomial(n=1,p=build_z3(z1))
        x = np.random.binomial(n=1,p=build_x(z1,z2,u))
        scm_np[i][0] = z1
        scm_np[i][1] = z2
        scm_np[i][2] = z3
        scm_np[i][3] = x
        sum = 0.0
        for j in range(d):
            if(i == 0):
                col_name.append('z4,'+str(j))
            z4 = np.random.binomial(n=1,p=build_z4(z2,z3,x))
            scm_np[i][j+4] = z4
            sum += -0.5*(2*z4 - 1)*para_c3()
        if(i==0):
            col_name.append('y')
        scm_np[i][d+4] = build_y(z1,z3,u,sum)
    print(scm_np)
    print('5')
    return scm_np,col_name


# Do Operator

In [ ]:
def causal_do(scm,new_x,d):
    scm_copy = scm.copy()
    scm_cols = np.size(scm_copy,axis=1)
    scm_rows = np.size(scm_copy,axis=0)
    scm_copy[:,3] = new_x
    for i in range(scm_rows):
        x = new_x[i]
        z1 = scm_copy[i][0]
        z2 = scm_copy[i][1]
        z3 = scm_copy[i][2]
        u = np.random.normal(loc = 0, scale = 2)
        sum = 0.0
        for j in range(d):
            z4 = np.random.binomial(n=1,p=build_z4(z2,z3,x))
            scm_copy[i][j+4] = z4
            sum += (-0.5*(2*z4 - 1)*para_c3())
        scm_copy[i][d+4] = build_y(z1,z3,u,sum)
    return scm_copy
def estimate_conditional_expectation1(df):
    scm_cols = np.size(df,axis=1)
    scm_rows = np.size(df,axis=0)
    a = df[:,3] == 0
    b = df[:,3] == 1
    y = df[:,scm_cols-1]

    p = np.mean(y[b])
    q = np.mean(y[a])
    delta = p-q
    return p,q,delta
def ab_test1(scm, n,d):
    n_a = int(n / 2)
    n_b = n - n_a
    set_variable = np.array([0]*n_a + [1]*n_b)
    scm = causal_do(scm,set_variable,d)
    return estimate_conditional_expectation1(scm)



# Weighted Generation

In [ ]:
def p_x(fd):
    df_copy = fd['x'].copy()
    size = fd.shape[0]
    pw2 = np.zeros(2)
    prob_w2 = np.zeros(size)
    for i in [0,1]:
        nm1 = df_copy == i
        pw2[i] = nm1.mean()
    for i in range(size):
        if df_copy[i] == 0:
            prob_w2[i] = pw2[0]
        else :
            prob_w2[i] = pw2[1]
    return prob_w2
def p_x_z1_z2_z3(fd):
    df_copy = fd.copy()
    size = fd.shape[0]
    x = df_copy['x']
    #print(df_copy.iloc[:,0:3])
    Z = df_copy.iloc[:,0:3].values.reshape(-1,3)

    clf = LogisticRegression(random_state=0).fit(Z,x)
    predict = clf.predict_proba(Z)
    proba_x_z = np.ones(size)
    for i in range(size):
        proba_x_z[i] *= predict[i,int(x[i])]
    return proba_x_z
def weightGenerator_1(df):
    data1 = p_x(df)
    data2 = p_x_z1_z2_z3(df)
    data_prob = np.divide(data1,data2)
    return data_prob


In [ ]:
def p_z4_z1_z2_z3(df):
    df_copy = df.copy()
    #print(df_copy.iloc[:,0:3])
    Z = df_copy.iloc[:,0:3].values.reshape(-1,3)
    #print(Z)
    proba_z4_z = np.ones(df.shape[0])
    for i in range(D):
        z4 = df[('z4,'+str(i))]
        #print(z4)
        clf = LogisticRegression(random_state=0).fit(Z,z4)
        predict = clf.predict_proba(Z)
        # print(predict)
        for j in range(df.shape[0]):
            proba_z4_z[j] *= predict[j,int(z4[j])]
        del clf
    return proba_z4_z
def p_z4_z1_z2_z3_x(df):
    df_copy = df.copy()
    #print(df_copy.iloc[:,0:4])
    Zx = df_copy.iloc[:,0:4].values.reshape(-1,4)
    proba_z4_zx = np.ones(df.shape[0])
    for i in range(D):
        z4 = df[('z4,'+str(i))]
        clf = LogisticRegression(random_state=0).fit(Zx,z4)
        predict = clf.predict_proba(Zx)
        for j in range(df.shape[0]):
            proba_z4_zx[j] *= predict[j,int(z4[j])]
        del clf
    return proba_z4_zx
def weightGenerator_2(df):
    data1 = p_z4_z1_z2_z3(df)
    data2 = p_z4_z1_z2_z3_x(df)
    data_prob = np.divide(data1,data2)
    return data_prob


# CWO

In [ ]:
def cwoBeta(X,y,weight,beta):
  Beta1 = LinearRegression().fit(X,y,sample_weight=weight)
  test_x=np.array([0, 1]).reshape((-1, 1))
  if beta==1:
    y_pred=Beta1.predict(test_x)
  else:
    y_pred=Beta1.predict(X)
  return y_pred
def weightedATE(df):
    data = df.copy()
    data_1 = data.drop(columns = ['x'])
    # print(data.shape)
    # print(data_1.shape)
    d = df.shape[1]-5
    #Z = data.iloc[:,4:4+d].values
    X = data['x'].values.reshape(-1,1)
    Y = data['y'].values.reshape(-1,1)
    Z = data_1.iloc[:,0:d+4].values.reshape(-1,d+4)
    sample_weight1 = weightGenerator_2(data)
    y_pred = cwoBeta(Z,Y,sample_weight1,2)
    sample_weight2 =weightGenerator_1(data)
    ate = cwoBeta(X,y_pred,sample_weight2,1)
    return ate[1],ate[0],ate[1]-ate[0]

# Neural Network

In [ ]:
hpDict={'conv_0_units': 40,
 'dropout': 0.2,
 'dropout_0_': 0.25,
 'input_units': 60,
 'learning_rate': 0.00925824604021082,
 'n_layers': 0}

In [ ]:
hpDict={'conv_0_units': 40,
 'dropout': 0.30000000000000004,
 'dropout_0_': 0.30000000000000004,
 'input_units': 80,
 'learning_rate': 0.00754819785170993,
 'n_layers': 1}

In [ ]:
hpDict={'conv_0_units': 10,
 'conv_1_units': 90,
 'conv_2_units': 10,
 'conv_3_units': 60,
 'dropout': 0.2,
 'dropout_0_': 0.4,
 'dropout_1_': 0.25,
 'dropout_2_': 0.4,
 'dropout_3_': 0.2,
 'input_units': 90,
 'learning_rate': 0.00033888747676374525,
 'n_layers': 0}

In [ ]:
hpDict={'conv_0_units': 30,
 'conv_1_units': 60,
 'dropout': 0.05,
 'dropout_0_': 0.4,
 'dropout_1_': 0.1,
 'input_units': 20,
 'learning_rate': 0.0006074808012923374,
 'n_layers': 2}

In [ ]:
def nnBeta(X,y,testX,weight,hp):
  d=X.shape[1]
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp['input_units'],activation='relu',input_shape=(d,)))
  Beta2.add(tf.keras.layers.Dropout(hp['dropout']))
  for i in range(hp['n_layers']):
    Beta2.add(tf.keras.layers.Dense(hp[f'conv_{i}_units'], activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp[f'dropout_{i}_']))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp['learning_rate']), loss='mse')

  early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=40, verbose=0, mode='auto', restore_best_weights=True)
  history=Beta2.fit(X,y,epochs=1000,
                    validation_split=0.1,
                    shuffle = True,
                    callbacks=[early_stop],
                    verbose=0
                    ,sample_weight=weight
                    )


  y_pred = Beta2.predict(testX)
  return y_pred

In [ ]:
def NeuralATE(df,hp):
  data = df.copy()
  d = df.shape[1]-5
  data_1 = data.drop(columns = ['x'])
  X = data['x'].values.reshape(-1,1)
  Y = data['y'].values.reshape(-1,1)
  Z = data_1.iloc[:,0:d+4].values.reshape(-1,d+4)
  sample_weight1 = weightGenerator_2(data)
  y_pred = nnBeta(Z,Y,Z,sample_weight1,hp)
  sample_weight2 =weightGenerator_1(data)
  test_x=np.array([0, 1]).reshape((-1, 1))
  ate = cwoBeta(X,y_pred,sample_weight2,1)
  return ate[1][0],ate[0][0],ate[1][0]-ate[0][0]

# mu and Dataframe saving

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/
%cd Datasets

/content/drive/My Drive
/content/drive/My Drive/Datasets


In [ ]:
D=10

In [ ]:
def doDo(D):
  N=10000000

  # c1=np.zeros(D)
  # c2=np.zeros(D)
  # c3=np.zeros(D)
  # for i in range(D):
  #     c1[i]=para_c1()
  #     c2[i]=para_c2()
  #     c3[i]=para_c3()
  scm_np , col_name = build_causal_model(N, D)
  df = pd.DataFrame(scm_np,columns=col_name)
  mu1,mu0,muATE=ab_test1(scm_np,N,D)

  return df,mu1,mu0

In [ ]:
fd,mu1,mu0=doDo(D)
print("Estimated ATE: {:.3f}".format(mu1-mu0))
print("mu(1): {:.3f}".format(mu1))
print("mu(0): {:.3f}".format(mu0))

In [ ]:
fd.to_pickle("drive/MyDrive/Datasets/mediator_dataframe_10.pkl")

In [ ]:
string1 = str(mu1) + " " + str(mu0)
print(string1)
f = open("drive/MyDrive/Datasets/mediator_mu_10.txt","w")
f.write(string1)
f.close()

# Result Generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
fd = pd.read_pickle("drive/MyDrive/Datasets/mediator_dataframe_4.pkl")
print(fd)
f = open('drive/MyDrive/Datasets/mediator_mu_4.txt','r')
mm = f.read()
mu1,mu0 = mm.split()
mu1=float(mu1)
mu0=float(mu0)
print(mu1,mu0)

# MAAE

In [ ]:

D=4

In [ ]:
def MAAEgenerator(db,mu1,mu0):
  muN1,muN0,_=NeuralATE(db,hpDict)
  muC1,muC0,_=weightedATE(db)
  maaeN=(abs(muN1-mu1)+abs(muN0-mu0))/2
  maaeC=(abs(muC1-mu1)+abs(muC0-mu0))/2
  return maaeN,maaeC


In [ ]:
for N_SAMPLE in range(500,10001,500):
  muN=np.zeros(100)
  muC=np.zeros(100)
  for k in range(100):
    print("K= ",k)
    db=fd.sample(n=N_SAMPLE).reset_index(drop=True)
    muN[k],muC[k]=MAAEgenerator(db,mu1,mu0)
    #print(k,muN[k],muC[k])
  MAAE_N=np.median(muN)
 # print("MAAE NN",MAAE_N)
  MAAE_C=np.median(muC)
  #print("MAAE CWO",MAAE_C)
  print(N_SAMPLE)
  print("MAAE NN: {:.5f}".format(MAAE_N))
  print("MAAE CWO: {:.5f}".format(MAAE_C))

# Hyper Parameter Tuning

In [ ]:
!pip install keras-tuner==1.0.0
import kerastuner

In [ ]:
from kerastuner.tuners import RandomSearch
import time
LOG_DIR = f"{int(time.time())}"

In [ ]:
def tempNN(hp):
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp.Int('input_units',
                                         min_value=10,
                                         max_value=150,
                                         step=10), activation='relu',input_shape=(7,)))
  Beta2.add(tf.keras.layers.Dropout(hp.Float('dropout',
                                             min_value=0.0,
                                             max_value=0.5,
                                             default=0.1,
                                             step=0.05,)))
  for i in range(hp.Int('n_layers', min_value=0, max_value=4,
                                         step=1)):
    Beta2.add(tf.keras.layers.Dense(hp.Int(f'conv_{i}_units',
                                           min_value=10,
                                           max_value=100,
                                           step=10), activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp.Float(f'dropout_{i}_',
                                             min_value=0.0,
                                             max_value=0.5,
                                             default=0.1,
                                             step=0.05,)))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate',
                                                                            min_value=1e-6,
                                                                            max_value=1e-2,
                                                                            sampling='LOG',
                                                                            default=1e-3)), loss='mse')

  return Beta2

In [ ]:
tuner = RandomSearch(
    tempNN,
    objective='val_loss',
    max_trials=3,  # how many model variations to test?
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

In [ ]:
data=fd.sample(n=4000).reset_index(drop=True)
d = fd.shape[1]-5
X=data.iloc[:,0:4+d].drop(columns = ['x']).values
Y = data['y'].values.reshape(-1,1)
sampleWeight = weightGenerator_2(data)


early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
tuner.search(X,Y,epochs=300,
             validation_split=0.2,
             shuffle = True,
             callbacks=[early_stop],
             verbose=0
             ,sample_weight=sampleWeight
         )

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'conv_0_units': 50,
 'conv_1_units': 60,
 'conv_2_units': 40,
 'conv_3_units': 90,
 'dropout': 0.0,
 'dropout_0_': 0.25,
 'dropout_1_': 0.35000000000000003,
 'dropout_2_': 0.25,
 'dropout_3_': 0.35000000000000003,
 'input_units': 70,
 'learning_rate': 0.002603236888594237,
 'n_layers': 0}

In [ ]:
hpDict=tuner.get_best_hyperparameters()[0].values

In [ ]:
hpDict={'conv_0_units': 50,
 'conv_1_units': 60,
 'conv_2_units': 30,
 'conv_3_units': 30,
 'dropout': 0.35000000000000003,
 'dropout_0_': 0.35000000000000003,
 'dropout_1_': 0.25,
 'dropout_2_': 0.15000000000000002,
 'dropout_3_': 0.1,
 'input_units': 100,
 'learning_rate': 0.00027492109691966345,
 'n_layers': 4}

In [ ]:
hpDict={'conv_0_units': 10,
 'dropout': 0.1,
 'dropout_0_': 0.1,
 'input_units': 140,
 'learning_rate': 2.471032658918893e-05,
 'n_layers': 1}

In [ ]:
hpDict={'dropout': 0.25,
 'input_units': 20,
 'learning_rate': 0.00082221764175021277,
 'n_layers': 0}

In [ ]:
hpDict={'conv_0_units': 50,
 'conv_1_units': 60,
 'conv_2_units': 40,
 'conv_3_units': 90,
 'dropout': 0.0,
 'dropout_0_': 0.25,
 'dropout_1_': 0.35000000000000003,
 'dropout_2_': 0.25,
 'dropout_3_': 0.35000000000000003,
 'input_units': 70,
 'learning_rate': 0.002603236888594237,
 'n_layers': 4}